# Class Metaprogramming
Class metaprogramming is the art of creating or customizing classes at runtime. Classes are first-class objects in Python, so a function can be used to create a new class at any time, without using the class keyword.

Metaclasses are powerful, but hard to get right. Class decorators solve many of the same
problems more simply.

## A Class Factory
The standard library has a class factory that we’ve seen several times in this book:
collections.namedtuple.

In [3]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError: # no .replace or .split
        pass # assume it's already a sequence of identifiers
    field_names = tuple(field_names)

    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
 
    def __iter__(self):
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)

    cls_attrs = dict(__slots__ = field_names,
                     __init__ = __init__,
                     __iter__ = __iter__,
                     __repr__ = __repr__)
    return type(cls_name, (object,), cls_attrs)


In [5]:
Dog = record_factory('Dog', 'name weight owner')

rex = Dog('Rex', 30, 'Bob')

In [6]:
rex

Dog(name='Rex', weight=30, owner='Bob')

type is a class. It behaves like a class that creates a new class when invoked with three arguments...

In [14]:
MySuperClass = object

MyClass = type('MyClass', (MySuperClass,), {'x': 42, 'x2': lambda self: self.x * 2})

In [16]:
MyClass.__dict__

mappingproxy({'x': 42,
              'x2': <function __main__.<lambda>(self)>,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

## A Class Decorator for Customizing Descriptors

In [6]:
import abc

class AutoStorage:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
 
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)


In [7]:
class Validated(abc.ABC, AutoStorage):
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    
    @abc.abstractmethod
    def validate(self, instance, value):
        """ return validated value or raise ValueError """


In [8]:
class Quantity(Validated):
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError("value must be > 0")
        return value


class NonBlank(Validated):
    """a string with at least one non-space character"""
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError("value cannot be empty or blank")
        return value


In [9]:
def entity(cls):
    for key, attr in cls.__dict__.items():
        if isinstance(attr, Validated):
            type_name = type(attr).__name__
            attr.storage_name = "_{}#{}".format(type_name, key)
    return cls


In [11]:
@entity
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price


In [12]:
raisins = LineItem("Golden raisins", 10, 6.95)

In [13]:
raisins.__dict__

{'_NonBlank#description': 'Golden raisins',
 '_Quantity#weight': 10,
 '_Quantity#price': 6.95}

A significant drawback of class decorators is that they act only on the class where they are directly applied. This means subclasses of the decorated class may or may not inherit the changes made by the decorator, depending on what those changes are. 

## What Happens When: Import Time Versus Runtime
At import time, the interpreter parses the source code of a .py module in one pass from top to bottom, and generates the bytecode to be executed. That’s when syntax errors may occur. If there is an up-to-date .pyc file available in the local __pycache__, those steps are skipped because the bytecode is ready to run.

The import statement is not merely a declaration but it actually runs all the top-level code of the imported module when it’s imported for the first time in the process.

In the previous paragraph, I wrote that importing "runs all the top-level code," but "toplevel code" requires some elaboration. The interpreter executes a def statement on the top level of a module when the module is imported, but what does that achieve? The interpreter compiles the function body (if it’s the first time that module is imported), and binds the function object to its global name, but it does not execute the body of the function, obviously.

For classes, the story is different: at import time, the interpreter executes the body of every class, even the body of classes nested in other classes. Execution of a class body means that the attributes and methods of the class are defined, and then the class object itself is built. In this sense, the body of classes is "top-level code": it runs at import time.

## Metaclasses 101
A metaclass is a class factory, except that instead of a function, like record_factory from Example 21-2, a metaclass is written as a class. Consider the Python object model: classes are objects, therefore each class must be an instance of some other class. By default, Python classes are instances of type. In other words, type is the metaclass for most built-in and user-defined classes:

In [15]:
'spam'.__class__

str

In [16]:
str.__class__  # instance of type

type

In [17]:
type.__class__

type

To avoid infinite regress, type is an instance of itself.

The classes object and type have a unique relationship: object is an instance of type, and type is a subclass of object. This relationship is "magic": it cannot be expressed in Python because either class would have to exist before the other could be defined. The fact that type is an instance of itself is also magical.

Every class is an instance of type, directly or indirectly, but only metaclasses are also subclasses of type. That’s the most important relationship to understand metaclasses: a metaclass, such as ABCMeta, inherits from type and has the power to construct classes.

***